# IMPORT DATA

In [2]:
import pandas as pd
import numpy as np
pd.set_option('chained_assignment', None)
import re
import math

#Turn chain assignments off:
pd.set_option('chained_assignment', None)

#PLUTO DATA for manhattan only
df = pd.read_csv("S:\\Office of Risk Management\\User Folders\\
Darius Mehri\\Teracata Address\\MN2.csv", low_memory=False)

df = df[["Block", "Lot", "Address", "NumFloors", "LotFront", 
         "LotDepth", "CB2010", "XCoord", "YCoord"]]


#replace address names to match dan's data:
df['Address'] = df['Address'].str.replace('ADAM C POWELL BLVD', '7 AVENUE')
#df['Address'] = df['Address'].str.replace('FREDRICK DOUGLASS BLVD', 'CENTRAL PARK WEST')
df['Address'] = df['Address'].str.replace('FREDRICK DOUGLASS BLVD', '8 AVENUE')
df['Address'] = df['Address'].str.replace('BROADWAY', 'BROADWAY AVENUE')
df['Address'] = df['Address'].str.replace('RIVERSIDE DRIVE', 'RIVERSIDE DRIVE AVENUE')
df['Address'] = df['Address'].str.replace('CENTRAL PARK WEST', '8 AVENUE')
#df['Address'] = df['Address'].str.replace('CENTRAL PARK WEST', 'CENTRAL PARK WEST AVENUE')
#df['Address'] = df['Address'].str.replace('CENTRAL PARK WEST AVENUE', '8TH AV')


#CENTRAL PARK WEST, 8TH AVE AND FRED DOUGLAS MUST BE CORRECTED


df["Address"] = df["Address"].astype(str)
df["CB2010"] = df["CB2010"].astype(str)

#df = df[df["CB2010"] != ""]

#dan's data
#dfd = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\MergedProject_RL_testAddress.csv")
#dfd = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\MergedProject_RL_wAddresstest.csv")
dfd = pd.read_csv("S:\\Office of Risk Management\\User Folders\\
Darius Mehri\\Teracata Address\\Formatted_ 1900 - 1940  Buildings Database_CC.csv")


dfd["ADDRESS, OWNER, ARCHITECT"] = dfd["ADDRESS, OWNER, ARCHITECT"].astype(str)
dfd["ADDRESS, OWNER, ARCHITECT"] = dfd["ADDRESS, OWNER, ARCHITECT"].map(str.upper)
dfd['ADDRESS, OWNER, ARCHITECT'] = dfd['ADDRESS, OWNER, ARCHITECT'].str.replace('CENTRAL PARK WEST', '8 AV')



it = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\Nodes_MN.csv", low_memory=False)

it["STNAME"] = it["STNAME"].map(str.strip)
it["STNAME"] = it["STNAME"].map(str.upper)

it["STNAME"] = it["STNAME"].str.replace('\r\n', '')
it['STNAME'] = it['STNAME'].str.replace('ADAM CLAYTON POWELL JR BLVD', '7 AVENUE')
#it['STNAME'] = it['STNAME'].str.replace('FREDERICK DOUGLASS BLVD', 'CENTRAL PARK WEST')
it['STNAME'] = it['STNAME'].str.replace('FREDERICK DOUGLASS BLVD', '8 AVENUE')
it['STNAME'] = it['STNAME'].str.replace('BROADWAY', 'BROADWAY AVENUE')
it['STNAME'] = it['STNAME'].str.replace('RIVERSIDE DR', 'RIVERSIDE DRIVE AVENUE')

it['STNAME'] = it['STNAME'].str.replace('W END', 'WEST END')
it['STNAME'] = it['STNAME'].str.replace('CENTRAL PARK W', '8 AVENUE')
#it['STNAME'] = it['STNAME'].str.replace('CENTRAL PARK WEST AVE', '8 AVE')



 


it = it[it.STNAME.str.contains(" LINE") == False]

it = it.reset_index(drop=True)




In [12]:
class TextClean:
    def __init__(self, dfd):
        self.dfd = dfd
        
        
    def CleanTheText(self):
        #CLEAN data
        #self.dfd = self.dfd[["ADDRESS, OWNER, ARCHITECT.1", "Number of Floors", "DESCRIPTION, COMMENTS"]]
        self.dfd = self.dfd[["ADDRESS, OWNER, ARCHITECT", "DESCRIPTION, COMMENTS"]]
        #self.dfd = self.dfd.rename(columns={'ADDRESS, OWNER, ARCHITECT.1': 'ADDRESS', 'Number of Floors': 'NumFloors', 'DESCRIPTION, COMMENTS': 'Comments'})
        self.dfd = self.dfd.rename(columns={'ADDRESS, OWNER, ARCHITECT': 'ADDRESS', 'DESCRIPTION, COMMENTS': 'Comments'})

        self.dfd["ADDRESS"] = self.dfd["ADDRESS"].astype(str)
        self.dfd["ADDRESS"] = self.dfd["ADDRESS"].map(str.upper)

        self.dfd["Street"] = ""
        self.dfd["Avenue"] = ""
        self.dfd["ftAve"] = ""
        self.dfd["SN"] = ""
        self.dfd["EW"] = ""
        self.dfd["Difference"] = "NA"

        
        count = 0
        
        
        countN = 0        
        for i in range(0, len(self.dfd)):
            if (" N S," in self.dfd["ADDRESS"][i] or " S S," in self.dfd["ADDRESS"][i]) and (" NO " not in  self.dfd["ADDRESS"][i]) and  \
            (";" not in  self.dfd["ADDRESS"][i]) and (" NOS " not in  self.dfd["ADDRESS"][i]) and (" S W " not in  self.dfd["ADDRESS"][i] \
            and ("-"  not in self.dfd["ADDRESS"][i]) ):
            
                count+=1
                #print self.dfd["ADDRESS"][i]
                
                if " N S " in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace(' N S ', ' N S,')
                if "ST N S" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('ST N S', 'ST, N S')  
                if "ST, ST" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('ST, ST', 'ST ST')
                if "ST S S" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('ST S S', 'ST, S S')
                if "163," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('163,', '163')
                if "120 W," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('120 W,', '120 W')
                if "115TH, ST," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('115TH, ST,', '115TH ST,')
                if "111TH ST, N S, 112TH ST, S S, 132 W LEXINGTON AV" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('111TH ST, N S, 112TH ST, S S, 132 W LEXINGTON AV', '111TH ST, N S, 132 W LEXINGTON AV')
                if "HOUSTON ST, EAST" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('HOUSTON ST, EAST', 'HOUSTON ST EAST')
                if "30TH ST, E," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('30TH ST, E,', '30TH ST E,')
                if "ST, E," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('ST, E,', 'ST E,')
                if "ST, WEST," in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('ST, WEST,', 'ST WEST,')
                if "1/2" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('1/2', '')
                if "1/4" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('1/4', '')
                if "3/4" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('3/4', '')
                if "1-3" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('1-3', '')
                if "&" in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace('&', '')
                if " OF " in self.dfd["ADDRESS"][i]:
                    self.dfd["ADDRESS"] = self.dfd["ADDRESS"].str.replace(' OF ', ' ')

                #split original, strip away owner, architect, etc
                a = self.dfd["ADDRESS"][i].split("|")
                self.dfd["ADDRESS"][i] = a[0]
                
                #98th st, n s, 100.10 e Madison av
                #strip white space, leading and trailing edge:
                self.dfd["ADDRESS"][i] = self.dfd["ADDRESS"][i].strip()
                

                #split the address
                sa = a[0].split(",")
                
                
                if "EIGHTEEN" in sa[len(sa)-1]:
                    sa.pop()
                if "REAR" in sa[len(sa)-1]:
                    sa.pop()
                if "INTERSECTION" in sa[len(sa)-1]:
                    sa.pop()
                if "YARD" in sa[len(sa)-1]:
                    sa.pop()
                if "BLOCK" in sa[len(sa)-1]:
                    sa.pop()
                if "?" in sa[len(sa)-1]:
                    sa.pop()
                    
                 
                #print sa

                #if (len(sa) == 3 or len(sa) == 4):
                self.dfd["Street"][i] = sa[0]
                #clean up strings, get rid of st, etc
                self.dfd["Street"][i] = re.sub('[STH]', '', self.dfd["Street"][i])

                #extract 133 street from 133D [133RD] ST
                z = self.dfd["Street"][i].split("D")
                self.dfd["Street"][i] = z[0]
                if ('NS' not in sa[1] and 'SS' not in sa[1] and 'N S' not in sa[1] 
                    and 'S S' not in sa[1] and ' N S' not in sa[1] and ' S S' not in sa[1]):
                    self.dfd["SN"][i] = sa[2]
                else:
                    self.dfd["SN"][i] = sa[1]

                #if ('NS' not in self.dfd["SN"][i] and 'SS' not in self.dfd["SN"][i] and 'N S' not in self.dfd["SN"][i] 
                    #and 'S S' not in self.dfd["SN"][i] and ' N S' not in self.dfd["SN"][i] and ' S S' not in self.dfd["SN"][i]):
                    #self.dfd["SN"][i] = sa[2]
                    #self.dfd["SN"] = sa[3]
                    #print sa[3]
                    #print self.dfd["SN"][i]
                    #a = self.dfd['ADDRESS'][i].split()
                    #self.dfd["SN"][i] = a[2]
                    #print self.dfd['ADDRESS'][i] + "new:"+ a[2]
                #problem address: HOUSTON ST, WEST, N S, CLARKSON ST, S S, 126 FT. WEST VARICK ST

                #example address: 45TH ST, N S, 413 E 11TH [ELEVENTH] AV
                #split avenue to get SN (south side, north side) and ftAve (feet from ave), and EW (east or west of ave)
                #b = sa[2].split()
                #98th st, n s, 100.10 e Madison av
                b = sa[len(sa)-1].split()
                
                if('BETWEEN'in b[0] or 'BET' in b[0] or 'EAST' in b[0]or 'BETWEEN'in b[0] or 'BET' in b[0] or 'EAST' in b[0]  ):
                    if (len(b)>3):
                        countN += 1
                    
  
                
                    
                self.dfd["ftAve"][i]= b[0]
                self.dfd["EW"][i]= b[1]
                
                if len(b) > 2:
                    self.dfd["Avenue"][i] = b[2]

                if len(b) == 4:
                    self.dfd["Avenue"][i] = self.dfd["Avenue"][i] + " " + b[3]

                if len(b) == 5:
                    self.dfd["Avenue"][i] = self.dfd["Avenue"][i] + " " + b[3] + " " + b[4]
                        
        #print self.dfd.head(20)
                #print "i:", i
                #print len(sa)-1
                #print b[0] + ', ' + b[1]
        #exportFile = self.dfd
        #exportFile.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\test.csv")
        #print  self.dfd["ftAve"] 
        #print 'this is the count:' + str(countN)

    #def ExportData(self):
    #    self.dfd.to_csv("C:\\Users\\ccamiolo\\Documents\\Full_Data_1900_1940\\test.csv")
    
    def FinalCleaning(self):
            
            #Remove all letters from ftAve
            self.dfd["ftAve"] = self.dfd["ftAve"].map(lambda x: x.strip(
                    'AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz'))
            
            #if (self.dfd["ftAve"]== "."):
                 #self.dfd["ftAve"] = self.dfd["ftAve"].map(lambda x: x.strip('.'))
            for i in range(0, len(self.dfd["ftAve"])):    
                var = "."
                if (var is self.dfd["ftAve"][i]):
                    self.dfd["ftAve"] = self.dfd["ftAve"].replace('.','')
                var2 = "51.10."
                if (var2 is self.dfd["ftAve"][i]):
                    self.dfd["ftAve"] = self.dfd["ftAve"].replace('0.','')
                #self.dfd["ftAve"] = df['result'].map(lambda x: x.rstrip('.'))
            #Remove all blanks from ftAve
            #for i in range(0, len(self.dfd['SN'])):    
                
                #if ('NS' not in self.dfd["SN"][i] and 'SS' not in self.dfd["SN"][i] and 'N S' not in self.dfd["SN"][i] 
                    #and 'S S' not in self.dfd["SN"][i] and ' N S' not in self.dfd["SN"][i] and ' S S' not in self.dfd["SN"][i]):
                    #self.dfd["SN"] = sa[3]
                    #print sa[3]
                    #print self.dfd["SN"][i]
                    #a = self.dfd['ADDRESS'][i].split()
                    #self.dfd["SN"][i] = a[2]
                    #print self.dfd['ADDRESS'][i] + "new:"+ a[2]
                        

            #self.dfd["ftAve"] = self.dfd["ftAve"].fillna(0)

            #Drop row based on element type:
            #self.dfd["ftAve"] = self.dfd["ftAve"].map(str.strip)

            #self.dfd["ftAve"] = self.dfd[self.dfd["ftAve"] != 0]
            
            
            
            #print self.dfd["ftAve"]
            
            #MORE CLEANING
            self.dfd["SN"]  = self.dfd["SN"].str.replace(' ', '')
            #strip white space from left and right
            self.dfd["Street"] = self.dfd["Street"].map(str.strip)
            self.dfd["Avenue"] = self.dfd["Avenue"].map(str.strip)
            self.dfd["ftAve"] = self.dfd["ftAve"].map(str.strip)
            self.dfd["SN"] = self.dfd["SN"].map(str.strip)
            self.dfd["EW"] = self.dfd["EW"].map(str.strip)



            #FINAL CLEANING

            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' ST', '')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('TH', '')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('2D', '2')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('3D', '3')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('1ST', '1')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('2ND', '2')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' AV', '')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('AV ', '')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('AVE A', 'A')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('B’WAY', 'BROADWAY')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('BWAY', 'BROADWAY')

            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('WADSWOR', 'WADSWORTH')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('.', '')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('MAIDSON', 'MADISON')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('EDGECOMBE ROAD', 'EDGECOMBE')

            for i in range(0, len(self.dfd)):
                if "[" in self.dfd["Avenue"][i]:
                    si = self.dfd["Avenue"][i]
                    si = si.split("[")
                    self.dfd["Avenue"][i] = si[0]


            #self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('WEST', 'WEST END')
            #self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('ST.', 'ST NICHOLAS')
            #self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('ST', 'ST NICHOLAS')
            #self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('FT', 'FORT WASHINGTON')
            #self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('FORT', 'FORT WASHINGTON')




            self.dfd = self.dfd[self.dfd["Street"] != 'NA']

            #self.dfd = self.dfd[self.dfd["Street"] != 'NA']

            self.dfd = self.dfd.reset_index(drop=True)

            for i in range (0, len(self.dfd)):
                st = self.dfd["Street"][i]
                st = ''.join(c for c in st if c.isdigit())
                self.dfd["Street"][i] = st
                #print st

            self.dfd = self.dfd[self.dfd["Street"] != '']

            self.dfd["Street"] = self.dfd["Street"].astype(int)



            self.dfd = self.dfd.reset_index(drop=True)

            self.dfd["ftAve"] = self.dfd["ftAve"].map(lambda x: x.strip('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))


            #CHANGE AVE A, B, C, ETC TO AVENUE A
            self.dfd["Avenue"] = " " + self.dfd["Avenue"] + " "

            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' A ', 'AVENUE A')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' B ', 'AVENUE B')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' C ', 'AVENUE C')
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace(' D ', 'AVENUE D')


            self.dfd = self.dfd.reset_index(drop=True)
            

            self.dfd["Avenue"] = self.dfd["Avenue"].astype(str)

            self.dfd["Avenue"] = self.dfd["Avenue"].map(str.strip)


            #IF DO NOT HAVE DISTANCE FROM AVE, CAN'T FIND ADDRESS, REMOVE
            self.dfd = self.dfd[self.dfd["ftAve"] != '']

            #REMOVE ALL ADDRESSES WHERE NUMBER OF FLOORS = 1, THESE BUILDINGS DO NOT EXIST IN MN ANYMORE
            #self.dfd = self.dfd[self.dfd["NumFloors"] != '1']

            self.dfd = self.dfd.reset_index(drop=True)


            #DETERMINE IF AVENUE IS ON EAST OR WEST SIDE
            self.dfd["AveEWSide"] = "NA"

            List_ave = ['1','2','3', 'LEXINGTON', 'PARK', 'MADISON']

            self.dfd["Avenue"] = self.dfd["Avenue"].astype(str)

            for i in range(0, len(self.dfd)):
                if self.dfd["Avenue"][i] in List_ave:
                    self.dfd["AveEWSide"][i] = "EAST"
                elif self.dfd["Avenue"][i] == '5':
                    self.dfd["AveEWSide"][i] = "5"
                else:
                    self.dfd["AveEWSide"][i] = "WEST"



            #SUBSET STREETS, AND WEST/EAST
            self.dfd = self.dfd[self.dfd["Street"] >= 15]
            #self.dfd = self.dfd[self.dfd["Street"] < 110]
            #self.dfd = self.dfd[self.dfd["AveEWSide"] != "EAST"]
            self.dfd = self.dfd[self.dfd["Avenue"] != 'AVENUE A']
            self.dfd = self.dfd[self.dfd["Avenue"] != 'AVENUE B']
            self.dfd = self.dfd[self.dfd["Avenue"] != 'AVENUE C']
            self.dfd = self.dfd[self.dfd["Avenue"] != 'AVENUE D']

            self.dfd = self.dfd.reset_index(drop=True)
            
            self.dfd["Street"] = self.dfd["Street"].astype(str)

            
            #replace old avenue names with correct ones
            self.dfd["Avenue"] = self.dfd["Avenue"].str.replace('CENTRAL PARK WEST', '8')
            self.dfd["Street"] = self.dfd["Street"].str.replace('124', 'LA SALLE')
            
            
            #print "length of dfd before subset for 13 and am:", len(self.dfd)
            #mask = (self.dfd["Street"] == '132') & (self.dfd["Avenue"] == 'AMSTERDAM')

            #self.dfd = self.dfd.loc[mask]
            
            #self.dfd= self.dfd[(self.dfd["Street"] == '132') | (self.dfd["Avenue"] == 'AMSTERDAM') ]

            
            #print "length of dfd after subset for 13 and am:", len(self.dfd)
            
            #self.dfd = self.dfd.reset_index(drop=True)


 
            

            self.dfd["AddressMatch"] = "NA"
            self.dfd["Error"] = "NA"

            #convert ftAve to integer

            self.dfd["ftAve"] = self.dfd["ftAve"].astype(float)
            
            for i in range(0, len(self.dfd)):
                    if self.dfd["ftAve"][i] != 'NA':
                        ftave = self.dfd["ftAve"][i]
                        ftave = round(ftave)
                        self.dfd["ftAve"][i] = ftave

            self.dfd["ftAve"] = self.dfd["ftAve"].astype(int)
            #print self.dfd.head(20)
            
            #self.dfd["Street"] = self.dfd["Street"].astype(str)
            self.dfd["ADDRESS"] = self.dfd["ADDRESS"].astype(str)
            self.dfd["AveEWSide"] = self.dfd["AveEWSide"].map(str.strip)
            
            global dfd2
            
            dfd2 = self.dfd
            
            return dfd2
                
            #self.dfd.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\test_final.csv")
            
    def GetBlockInt(self, df):
        for z in range(0,len(self.dfd)):
        #for z in range(297,300):
            #Create lists for street and avenue address
            #print ""
            #print "*******************************************"
            #print "*******************************************"
            #print "z= ", z
            
            if z%50 == 0:
                print "z= ", z

            
            global list_a
            list_b = []
            list_a = []

            street = self.dfd["Street"][z]
            street = street + " STREET"

            avenue = self.dfd["Avenue"][z]
            avenue = avenue + " AVE"

            SNSide = self.dfd["SN"][z]
            EWSide = self.dfd["EW"][z]
            AveOnEWSide = self.dfd["AveEWSide"][z]


            
            
            #get street and avenue instersection in pluto data
            count = 0
            for i in range(0, len(df)):
                if street in df["Address"][i]:
                    list_a.append([df["Block"][i], df["Address"][i], df["NumFloors"][i],df["LotFront"][i], df["LotDepth"][i], df["CB2010"][i],df["XCoord"][i], df["YCoord"][i] ])
                if avenue in df["Address"][i]:
                    list_b.append([df["Block"][i], df["Address"][i], df["NumFloors"][i],df["LotFront"][i], df["LotDepth"][i],df["CB2010"][i], df["XCoord"][i], df["YCoord"][i] ])
                count+=1

            #print count
            #print "list a", list_a
            #print ""
            #print "list b", list_b
            #print ""
            
            #use lists above to create list of all addresses where blocks intersect, should end up
            #with a maximum of four unique blocks and their addresses
            final_4 = []

            for i in range(0, len(list_a)):
                for j in range(0, len(list_b)):
                    #are blocks equal?
                    if list_a[i][0] == list_b[j][0]:
                        #print list_a[i][0], list_b[j][0]
                        final_4.append([list_a[i][0], list_a[i][1], list_a[i][2],list_a[i][3], list_a[i][4], list_a[i][5],list_a[i][6],list_a[i][7] ])
                        final_4.append([list_b[j][0], list_b[j][1], list_b[j][2], list_b[j][3], list_b[j][4],  list_b[j][5],list_b[j][6], list_b[j][7] ])

            #print "length of final 4 ",len(final_4)             

            #CREATE DATAFRAME FROM LIST
            global df2
            df2 = pd.DataFrame(final_4, columns=['Block', 'Address', 'NumFloors', 'LotFront', 'LotDepth', 'CB2010', 'X', 'Y'])
            
            
            
            if not df2.empty:
                #print "dataframe 1", df2

                #drop duplicates
                df2 = df2.drop_duplicates(subset=['Block', 'Address']).reset_index(drop=True)

                #subset for north or south side of the street
                df2["HouseNumber"] = "NA"
                df2["OddEven"] = "NA"


                for i in range(0, len(df2)):
                    a = df2["Address"][i].split()
                    st = a[0]
                    st = ''.join(c for c in st if c.isdigit())
                    df2["HouseNumber"][i] = st


                #drop house numbers with NA  and blank     
                df2 = df2[df2["HouseNumber"] != 'NA']

                df2 = df2[df2["HouseNumber"] != '']

                df2 = df2.reset_index(drop=True)

                df2["HouseNumber"] = df2["HouseNumber"].astype(int)

                for i in range(0, len(df2)):
                    if df2["HouseNumber"][i]%2 == 0:
                        df2["OddEven"][i] = "Even"
                    else:
                        df2["OddEven"][i] = "Odd"

                #print ""
                #print "original df2"
                #print df2

                df2ave = df2[df2.Address.str.contains("AVENUE") == True]

                df2st = df2[df2.Address.str.contains("STREET") == True]



                #GET THE X Y INTERSECTION POINT
                list_aInt = []
                list_bInt = []


                street = self.dfd["Street"][z]
                street = street + " ST"


                avenue = self.dfd["Avenue"][z]
                avenue = avenue + " AVE"

                #print ""
                #print "street"
                #print street
                #print "avenue"
                #print avenue

                for i in range(0, len(it)):
                    if street in it['STNAME'][i]:
                        list_aInt.append([it["NODEID"][i], it["STNAME"][i], it["X"][i],it["Y"][i]])
                    if avenue in it['STNAME'][i]:
                        list_bInt.append([it["NODEID"][i], it["STNAME"][i], it["X"][i],it["Y"][i]])


                final_4Int = []

                for i in range(0, len(list_aInt)):
                    for j in range(0, len(list_bInt)):
                        #are blocks equal?
                        if list_aInt[i][0] == list_bInt[j][0]:
                            #print list_a[i][0], list_b[j][0]
                            final_4Int.append([list_aInt[i][0], list_aInt[i][1], list_aInt[i][2],list_aInt[i][3]])
                            final_4Int.append([list_bInt[j][0], list_bInt[j][1], list_bInt[j][2], list_bInt[j][3]])


                global df2Int
                df2Int = pd.DataFrame(final_4Int, columns=['NODEID', 'STNAME', 'X', 'Y'])
                
                if not df2Int.empty:
                    #df2Int = df2Int.drop_duplicates(subset=['STNAME', 'X', 'Y']).reset_index(drop=True)


                    #print "final4Int"
                    #print final_4Int
                    #print "df2Int"
                    #print df2Int




                    #SUBSET DF2 BASED ON THE X, Y COORDINATES

                    xy = [int(df2Int["X"][0]), int(df2Int["Y"][0])]


                    df2['X'] = df2['X'].astype(float)
                    df2['Y'] = df2['Y'].astype(float)


                    #print ""
                    #print "xy[0-1]", xy[0], xy[1]
                    #print ""


                    #print ""
                    #print "df2ave"
                    #print df2ave
                    #print ""
                    #print "df2st"
                    #print df2st

                    #subset west, east, north south to get correct block
                    if dfd2["AveEWSide"][z] =="WEST":
                        #X Coordinate decreases as go from east to west
                        if dfd2["EW"][z] == "W": 
                            df2st = df2st[df2st.X < xy[0]]
                            #need to subset for side of street, it not will have both sides of street
                            df2ave = df2ave[df2ave.X < xy[0]]
                            df2st = df2st.reset_index(drop=True)
                        else:
                            df2st = df2st[df2st['X'] > xy[0]]
                            df2ave = df2ave[df2ave.X > xy[0]]
                            df2st = df2st.reset_index(drop=True)

                        #Y coord increases as go south to north, add 40 to make sure grabbing building
                        if "SS" in dfd2["SN"][z]:
                            #df2ave = df2ave[df2ave.Y <= xy[1] + 40]
                            df2ave = df2ave[df2ave.Y <= xy[1]]

                        else:
                            #df2ave = df2ave[df2ave.Y >= xy[1] - 40]
                            df2ave = df2ave[df2ave.Y >= xy[1]]




                    #subset for north or south side of street
                    if dfd2["SN"][z] =="SS":
                        df2st = df2st[df2st['OddEven'] == 'Even']
                    else:
                        df2st = df2st[df2st['OddEven'] == 'Odd']

                    df2st = df2st.reset_index(drop=True)


                    #sort avenue to get corner building, avenues go north and south
                    if "SS" in dfd2["SN"][z]:
                        df2ave = df2ave.sort_values(['HouseNumber'], ascending=[False])
                        df2ave = df2ave.reset_index(drop=True)
                    else: 
                        df2ave = df2ave.sort_values(['HouseNumber'], ascending=[True])
                        df2ave = df2ave.reset_index(drop=True)

                    #print ""
                    #print "df2ave sorted"
                    #print df2ave


                    #Get lot depth 
                    global addLotDepth
                    addLotDepth = 0


                    if not df2ave.empty:
                        addLotDepth = df2ave["LotDepth"][0]
                    else:
                        addLotDepth = 0



                    #print ""
                    #print "add lot depth"
                    #print addLotDepth

                    df2 = df2st

                    #print ""
                    #print df2

                    data.GetAddressMatch(df2, z, addLotDepth, df2ave)
        
            else:
                dfd2["AddressMatch"][z] = "NO INTERSECTION"
        
        else:
            dfd2["AddressMatch"][z] = "NO INTERSECTION"
     
    
    def GetAddressMatch(self,df2, z, addLotDepth, df2ave):
        df2["Distance"] = 'NA'
        df2["EucDistance"] = 'NA'
        df2["PerfectMatch"] = "NA"
        df2["Difference"] = 'NA'


        #calculate distance and euclidean distance, e distance is defined as absolute value of distance of each address
        #minus the distance of address searching for, the address with the smallest euclidian distance is the best match, distances
        #are in feet




        dfd2["ftAve"][z] = int(dfd2["ftAve"][z])

        dist = dfd2["ftAve"][z]
        #print "dist", dist

        df2 = df2.reset_index(drop=True)
        #print df2
            
        #sort if either east or west to get into proper order to count lot frontage
        
        
        if dfd2["AveEWSide"][z] == 'WEST':
            if "W" in dfd2["EW"][z]:
                df2 = df2.sort_values(['HouseNumber'], ascending=[True])
            else: 
                #print "SORTING FOR EAST!!!"
                df2 = df2.sort_values(['HouseNumber'], ascending=[False])
         
        elif dfd2["AveEWSide"][z] == 'EAST':
            if "W" in dfd2["EW"][z]:
                df2 = df2.sort_values(['HouseNumber'], ascending=[False])
            else: 
                #print "SORTING FOR EAST!!!"
                df2 = df2.sort_values(['HouseNumber'], ascending=[True])
        
        #elif dfd2["AveEWSide"][z] == '5':
        else:
            df2 = df2.sort_values(['HouseNumber'], ascending=[True])
        
        
        df2 = df2.reset_index(drop=True)
            
            
        #ADD FAKE LOT 
        
        if not df2ave.empty:
            df2.loc[-1] = ['', df2ave["Address"][0], '', addLotDepth, '','', '', '', '', '', "", "", "", ""]  # adding a row
        else:
            df2.loc[-1] = ['', 'FAKE', '', addLotDepth, '','', '', '', '', '', "", "", "", ""]  # adding a row
        
        
        #df2.loc[-1] = ['', df2ave["Address"][0], '', addLotDepth, '','', '', '', '', '', "", "", "", ""]  # adding a row
        df2.index = df2.index + 1  # shifting index
        df2 = df2.sort_index()  # sorting by index
        
        
        
        
        #PROBLEM IS SHOULD NOT BE ADDING LOT FRONT FOR THE FIRST IF HAVE ADDLOTDEPTH > 0, IF LOT DEPTH
        #IS ZERO THEN ADD
            
            
        for i in range(0, len(df2)):
            if i == 0:
                df2["Distance"][0] = df2["LotFront"][0]
                eucDist = df2["Distance"][i] - dist
                df2["Difference"][i] = eucDist
                df2["EucDistance"][i] = abs(eucDist)
                   

            #print df2["Distance"][i]
            if i>0:
                df2["Distance"][i] = df2["LotFront"][i] + df2["Distance"][i-1]
                eucDist = df2["Distance"][i] - dist
                df2["Difference"][i] = eucDist
                df2["EucDistance"][i] = abs(eucDist)
                    
                    
        for i in range(0, len(df2)):
            if df2["EucDistance"][i] != 'NA':
                eucDistance = df2["EucDistance"][i]
                eucDistance = round(eucDistance)
                df2["EucDistance"][i] = eucDistance
    
        df2 = df2.reset_index(drop=True)
        
        #***********************************
        # CORRECT IF EUCLIDIEAN DISTANCE IS GREATER THAN 20, GET RIDE OF FIRST ROW WITH FAKE ADDRESS THAT HAD LOT DEPTH
        if (df2["EucDistance"].min() > 20 and len(df2) > 1):
            #df2 = df2.drop(df2.head(1).index, inplace=True)
            #df2 = df2.drop(df2.index[[0]])
            #df2 = df2.reset_index(drop=True)
            
            df2 = df2.drop(df2.index[[0]])
            #df2 = df2.drop('index', 1)
            df2 = df2.reset_index(drop=True)

            for i in range(0, len(df2)):
                if i == 0:
                    df2["Distance"][0] = df2["LotFront"][0]
                    eucDist = df2["Distance"][i] - dist
                    df2["Difference"][i] = eucDist
                    df2["EucDistance"][i] = abs(eucDist)


                #print df2["Distance"][i]
                if i>0:
                    df2["Distance"][i] = df2["LotFront"][i] + df2["Distance"][i-1]
                    eucDist = df2["Distance"][i] - dist
                    df2["Difference"][i] = eucDist
                    df2["EucDistance"][i] = abs(eucDist)
                    
                    
            for i in range(0, len(df2)):
                if df2["EucDistance"][i] != 'NA':
                    eucDistance = df2["EucDistance"][i]
                    eucDistance = round(eucDistance)
                    df2["EucDistance"][i] = eucDistance
            
        

        #create new data frame so can sort
        global df3
        df3 = df2[["Block", "Address", "NumFloors", "LotFront", "HouseNumber", "OddEven", "Distance","X", "Y", "EucDistance", "Difference", "PerfectMatch"]]
        #print df2
        #sort to get the smallest distance, best match
        df3 = df3.sort_values(by = 'EucDistance', ascending=True).reset_index(drop=False)
        df3 = df3.reset_index(drop=True)
        df3 = df3.drop('index', 1)



        #INSERT MATCHED ADDRESS INTO DAN'S DATAFRAME
        global AMatch
        AMatch = dfd2
        AMatch["Building Status"] = ""
        #get the next building over so need df3["Address"][1] not f3["Address"][0]
            
        #get the index in the dataframe where euclidian distance is zero plus 1, the next address is the exact match
        #print "df2 address match"
        #print df2
        indexAdM = df2['EucDistance'].argmin() + 1
        #print ""
        #print "indexAdM"
        #print indexAdM
        #print "length of df2"
        #print len(df2)
        
        #to keep from index going out of range
        if indexAdM == len(df2):
            AMatch["AddressMatch"][z] = df2["Address"][len(df2) - 1]
        else:
            AMatch["AddressMatch"][z] = df2["Address"][indexAdM]
        
        #correct for if error not equal to zero, if overshoot then choose next building
        #if undershoot choose building
        
        
        #print "EUCLIDEAN DISTANCE DF3"
        #print "DISTANCE"
        #print dist
        #print ""
        #print df2
        
        if df3["EucDistance"][0] != 0:
            AMatch["AddressMatch"][z] = df3["Address"][0]
            
        
        AMatch["Error"][z] = df3["EucDistance"][0]
        AMatch["Difference"][z] = df3["Difference"][0]
        
        #if the distance from the avenue is less than the lot depth for the corner building, then it is an avenue address
        if dist < (addLotDepth - 5):
            AMatch["AddressMatch"][z] = df2ave["Address"][0]
            AMatch["Error"][z] = 'NA'
            AMatch["Difference"][z] = 0
            
    
    
        
    def BuildingStatus(self, AMatch):
        for i in range(0, len(AMatch)):
            if AMatch["Error"][i] < 11:
                AMatch["Building Status"][i] = "Exists"
            if AMatch["Error"][i] > 10:
                AMatch["Building Status"][i] = "Unknown"
            if "AVENUE" in AMatch["AddressMatch"][i]:
                AMatch["Building Status"][i] = "Probably Demolished"
            if AMatch["Error"][i] == 'NA':
                AMatch["Building Status"][i] = "Probably Demolished"
            if "NO INTERSECTION" in AMatch["AddressMatch"][i]:
                AMatch["Building Status"][i] = "Probably Demolished"
            if AMatch["AddressMatch"][i] == "FAKE":
                AMatch["Building Status"][i] = "Unknown"
     
    
data = TextClean(dfd)
print "Calling clean the text"
data.CleanTheText()
print "Calling final text cleaning"
data.FinalCleaning()
print "Get the Address"
data.GetBlockInt(df)

print "Get building status"
data.BuildingStatus(AMatch)

print "Done"

#print "Exporting data"
#data.ExportData()




Calling clean the text
Calling final text cleaning
Get the Address
z=  0
z=  50
z=  100
z=  150
z=  200
z=  250
z=  300
z=  350
z=  400
z=  450
z=  500
z=  550
z=  600
z=  650
z=  700
z=  750
z=  800
z=  850
z=  900
z=  950
z=  1000
z=  1050
z=  1100
z=  1150
z=  1200
z=  1250
z=  1300
z=  1350
z=  1400
z=  1450
z=  1500
z=  1550
z=  1600
z=  1650
z=  1700
z=  1750
z=  1800
z=  1850
z=  1900
z=  1950
z=  2000
z=  2050
z=  2100
Get building status
Done


In [14]:
AMatch

,ADDRESS,Comments,Street,Avenue,ftAve,SN,EW,Difference,AveEWSide,AddressMatch,Error,Building Status
0,"<124TH>, S S, 175 E BROADWAY. SEE 123D [123RD]...",NaN,LA SALLE,BROADWAY,175,SS,E,-175,WEST,FAKE,175,Unknown
1,"100TH ST, N S, 100 E 2D [SECOND] AV",NaN,100,2,100,NS,E,0,EAST,1924 2 AVENUE,NA,Probably Demolished
2,"123D [123RD] ST, N S, 190 E PARK AV",NaN,123,PARK,190,NS,E,-100,EAST,1761 PARK AVENUE,100,Probably Demolished
3,"116TH ST, S S, 325 E PLEASANT AV",NaN,116,PLEASANT,325,SS,E,-325,WEST,FAKE,325,Unknown
4,"124TH ST, N S, 225 W 1ST [FIRST] AV",NaN,LA SALLE,1,225,NS,W,NA,EAST,NO INTERSECTION,NA,Probably Demolished
5,"118TH ST, S S, 360 E LENOX AV",NaN,118,LENOX,360,SS,E,-275,WEST,160 LENOX AVENUE,275,Probably Demolished
6,"126TH ST, S S, 125 W AMSTERDAM AV",NaN,126,AMSTERDAM,125,SS,W,-11,WEST,504 WEST 126 STREET,11,Unknown
7,"44TH ST, S S, 100 E 8TH [EIGHTH] AV",NaN,44,8,100,SS,E,0,WEST,310 WEST 144 STREET,0,Exists
8,"116TH ST, N S, 143.8 W ST NICHOLAS AV",NaN,116,ST NICHOLAS,144,NS,W,-0.33,WEST,215 WEST 116 STREET,0,Exists
9,"115TH ST, N S, 215 E 1ST [FIRST] AV",NaN,115,1,215,NS,E,-20,EAST,313 EAST 115 STREET,20,Unknown


In [15]:
AMatch.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Teracata Address\\EASEMENT_STREET_MATCH.csv")